In [ ]:
%matplotlib widget
import numpy as np
from scipy.fft import fft, ifft
import matplotlib.pyplot as plt
from Functions.signal_generator import generate_ADSB, digitize_signal, correlate_signals, \
                                       normalize_signal, signal_start_pause_length, filter_with_window
from preambules_list import Preambule, preambule_list
from Functions.utils import SAMPLE_ADSB_BYTES

class ReverseCorrSampleBase:
    def __init__(self, correlation: np.ndarray, name: str, signal: np.ndarray):
        self.correlation = correlation
        self.signal = signal
        self.result_mask = np.zeros(len(correlation))
        self.name = name
        self.result_correlation = np.zeros(len(correlation))

    def get_name(self) -> str:
        return self.name

    # Does deconvolution
    def do_reverse_correlation(self) -> np.ndarray:
        if len(self.correlation) != len(self.signal):
            raise ValueError(f"Correlation and signal must have the same length: {len(self.correlation)} != {len(self.signal)}")
        R_f = fft(self.correlation)
        X_f = fft(self.signal, len(self.correlation))

        # Regularization to avoid division by zero
        eps = 1e-10
        Y_f = R_f / (X_f + eps)

        self.result_mask = np.real(ifft(Y_f))

        # Remove outlying peaks from result_mask
        threshold = np.mean(self.result_mask) + 3 * np.std(self.result_mask)
        self.result_mask[self.result_mask > threshold] = threshold
        self.result_mask[self.result_mask < -threshold] = -threshold

        # extended = np.concatenate((np.zeros(100), self.signal))
        self.result_correlation = correlate_signals(self.signal, self.result_mask[:75])

    def display_reverse_corr(self):
        plt.figure(figsize=(8, 8))
        plt.suptitle(self.name)
        plt.subplot(411)
        plt.title("Signal")
        plt.stairs(self.signal)
        plt.ylabel("Amplitude")
        plt.subplot(412)
        plt.title("Correlation")
        plt.stairs(self.correlation)
        plt.ylabel("Amplitude")
        plt.subplot(413)
        plt.title("Mask")
        plt.stairs(self.result_mask)
        plt.ylabel("Amplitude")
        plt.xlabel("Sample, n")
        plt.subplot(414)
        plt.title("Result correlation")
        plt.stairs(self.result_correlation)
        plt.ylabel("Amplitude")
        plt.xlabel("Sample, n")
        plt.tight_layout()
        plt.show()

class ReverseCorrWindowed(ReverseCorrSampleBase):
    def __init__(self, initial_corr: np.ndarray, name: str, signal: np.ndarray, window_width: int, expected_max: int):
        correlation, _ = filter_with_window(initial_corr, window_width, expected_max)
        super().__init__(correlation, name, signal)

example_correlation = np.array([0, 0, 0, 0, 1, 2, 3, 2, 1])
example_signal = np.array([0, 0, 0, 1, 1, 1, 0, 0, 0])
reverse_corr_example = ReverseCorrSampleBase(example_correlation, "Example", example_signal)
reverse_corr_example.do_reverse_correlation()
reverse_corr_example.display_reverse_corr()

In [ ]:
def cutout_signals(signal: np.ndarray, preamble: np.ndarray):
    signal = signal[:1100]

    signal_corr = correlate_signals(signal, np.repeat(preamble.get_coefficients(), 1))

    CUTOFF_REGION_LOW = 0
    CUTOFF_REGION_HIGH = 300
    display_signal = signal[CUTOFF_REGION_LOW:CUTOFF_REGION_HIGH]
    signal_corr = signal_corr[CUTOFF_REGION_LOW:CUTOFF_REGION_HIGH]
    return display_signal, signal_corr

#Configuration
amplitude = 1
preamble = preambule_list[Preambule.Ideal.value]
expected_maximum = signal_start_pause_length + len(preamble.get_coefficients())
#Generate ADS-B signal
ideal_signal, filtered_signal, noisy_signal = generate_ADSB(amplitude, SAMPLE_ADSB_BYTES, generate_bits=False)
noisy_signal, _ = digitize_signal(noisy_signal, 100E6, 10E6, 1.4, 2**10)
ideal_signal, _ = digitize_signal(ideal_signal, 100E6, 10E6, 1.4, 2**10)

# Assemble needed signals
cutoff_noisy_signal, cutoff_noisy_corr = cutout_signals(noisy_signal, preamble)
cutoff_ideal_signal, cutoff_ideal_corr = cutout_signals(ideal_signal, preamble)

impulse_corr = np.concatenate((np.zeros(expected_maximum), np.ones(1)))
impulse_corr = np.concatenate((impulse_corr, np.zeros(len(cutoff_noisy_signal) - len(impulse_corr))))

sinc_x = np.linspace(-expected_maximum, len(cutoff_noisy_signal) - expected_maximum, len(cutoff_noisy_signal))
sinc_corr = np.sinc(sinc_x / np.pi)
sinc_corr = sinc_corr / np.max(np.abs(sinc_corr)) * np.max(np.abs(cutoff_noisy_signal))

hamming_corr, centered_hamming_window = filter_with_window(cutoff_noisy_corr, 100, expected_maximum)
hamming_corr_2, centered_hamming_window_2 = filter_with_window(cutoff_noisy_corr, 200, expected_maximum)
hamming_corr_3, centered_hamming_window_3 = filter_with_window(cutoff_noisy_corr, 500, expected_maximum)

plt.figure()
plt.plot(normalize_signal(cutoff_noisy_corr), label = "Normal")
plt.plot(normalize_signal(centered_hamming_window), label = "Hamming 1")
plt.plot(normalize_signal(centered_hamming_window_2), label = "Hamming 2")
plt.plot(normalize_signal(centered_hamming_window_3), label = "Hamming 3")
plt.plot(normalize_signal(hamming_corr), label = "With hamming 1")
plt.legend()
plt.show()

#Insert signals into appropriate arrays for execution
reverse_corr_samples = [
    ReverseCorrSampleBase(cutoff_ideal_corr, "Ideal normal", cutoff_ideal_signal),
    ReverseCorrWindowed(cutoff_ideal_corr, "Ideal hamming 1", cutoff_ideal_signal, 100, expected_maximum),
    ReverseCorrWindowed(cutoff_ideal_corr, "Ideal hamming 2", cutoff_ideal_signal, 10000, expected_maximum),
    ReverseCorrSampleBase(cutoff_noisy_corr, "Normal", cutoff_noisy_signal),
    ReverseCorrWindowed(cutoff_noisy_corr, "With hamming 1", cutoff_noisy_signal, 100, expected_maximum),
    ReverseCorrWindowed(cutoff_noisy_corr, "With hamming 2", cutoff_noisy_signal, 200, expected_maximum),
    ReverseCorrWindowed(cutoff_noisy_corr, "With hamming 2", cutoff_noisy_signal, 200, expected_maximum),
    ReverseCorrWindowed(cutoff_noisy_corr, "With hamming 3", cutoff_noisy_signal, 500, expected_maximum),
    ReverseCorrSampleBase(impulse_corr, "Impulse", cutoff_noisy_signal),
    ReverseCorrSampleBase(sinc_corr, "Sinc", cutoff_noisy_signal),
]

for i in range(len(reverse_corr_samples)):
    reverse_corr_sample = reverse_corr_samples[i]
    reverse_corr_sample.do_reverse_correlation()
    reverse_corr_sample.display_reverse_corr()
    # TODO: artimiausia lapeli mazinti

In [ ]:
#TODO: du variantai, vienas kuris ant duomenu neislenda auksciau preambules, kitas kuris neturi peako pries preambule